In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
from autokeras.keras_layers import ExpandLastDim
from autokeras.keras_layers import CastToFloat32
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, LSTM, SimpleRNN
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.layers.experimental.preprocessing import RandomTranslation, RandomFlip
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.applications import EfficientNetB7
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

# Make Input

In [3]:
def calculateWeight(tlx):
    tlx_weight = {'Mental':[0], 
                  'Physical':[0], 
                  'Temporal':[0], 
                  'Effort':[0],
                  'Performance':[0],
                  'Frustration':[0],
                  'Sum':[0]}
    tlx_weight = pd.DataFrame(tlx_weight)
    for i in range(len(tlx)):
        score = [0,0,0,0,0,0,0]
        for col1 in range(1,len(tlx.columns)):
            for col2 in range(col1+1, len(tlx.columns)):
                if tlx[tlx.columns[col1]][i] > tlx[tlx.columns[col2]][i]:
                    score[col1-1]+=1
                elif tlx[tlx.columns[col1]][i] < tlx[tlx.columns[col2]][i]:
                    score[col2-1]+=1
                else :
                    score[col1-1]+=0.5
                    score[col2-1]+=0.5
                    
        score[6] = score[0]+score[1]+score[2]+score[3]+score[4]+score[5]
        tlx_weight.loc[i]=score
    #print(tlx_weight.loc[0])
    return tlx_weight

def calculate_tlxLevel(tlx, tlx_weight):
    result = {'Mental':[0], 
                  'Physical':[0], 
                  'Temporal':[0], 
                  'Effort':[0],
                  'Performance':[0],
                  'Frustration':[0],
                  'Score':[0]}
    result = pd.DataFrame(result)
    for i in range(len(tlx)):
        score = [0,0,0,0,0,0,0]
        for col in range(len(tlx_weight.columns)-1):
            score[col] = int(tlx[tlx.columns[col+1]].loc[i] * tlx_weight[tlx_weight.columns[col]].loc[i] )
        score[6] =int((score[0]+score[1]+score[2]+score[3]+score[4]+score[5] )/ tlx_weight[tlx_weight.columns[6]].loc[i]/10 + 0.5)
        if score[6]>10: score[6]=10
        if score[6]<0: score[6]=0
        result.loc[i]=score
    return result['Score']

In [7]:
src = 'EEG data/User/'
files = os.listdir(src)
tlx=[]
for f in files:
    tlx.append(pd.read_csv(src+f))
src = 'EEG data/prepross/'
datas = os.listdir(src)
workloadLevel = []
for t in tlx:
    workloadLevel.append(calculate_tlxLevel(t, calculateWeight(t)))
eegData=[]
for d in datas:
    eegData.append(pd.read_csv(src+d))

In [38]:
def cal_theta_alpha(df):
    col = ['POW.AF3.Theta', 'POW.AF3.Alpha', 'POW.F7.Theta', 'POW.F7.Alpha', 
           'POW.F3.Theta', 'POW.F3.Alpha', 'POW.FC5.Theta', 'POW.FC5.Alpha', 
           'POW.T7.Theta', 'POW.T7.Alpha', 'POW.P7.Theta', 'POW.P7.Alpha',
           'POW.O1.Theta', 'POW.O1.Alpha', 'POW.O2.Theta', 'POW.O2.Alpha', 
           'POW.P8.Theta', 'POW.P8.Alpha', 'POW.T8.Theta', 'POW.T8.Alpha',
           'POW.FC6.Theta', 'POW.FC6.Alpha',  'POW.F4.Theta', 'POW.F4.Alpha', 
           'POW.F8.Theta', 'POW.F8.Alpha','POW.AF4.Theta', 'POW.AF4.Alpha','MarkerValueInt']
    col_rename = ['AF3.Theta', 'AF3.Alpha', 'F7.Theta','F7.Alpha', 
                               'F3.Theta', 'F3.Alpha', 'FC5.Theta', 'FC5.Alpha', 
                               'T7.Theta', 'T7.Alpha', 'P7.Theta', 'P7.Alpha', 
                               'O1.Theta', 'O1.Alpha', 'O2.Theta', 'O2.Alpha',
                               'P8.Theta', 'P8.Alpha', 'T8.Theta','T8.Alpha',
                               'FC6.Theta', 'FC6.Alpha', 'F4.Theta', 'F4.Alpha', 
                               'F8.Theta', 'F8.Alpha', 'AF4.Theta', 'AF4.Alpha','vis_name']
    col2 = ['AF3', 'F7','F3', 'FC5', 'T7', 'P7', 'O1', 'O2',
            'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4','vis_name']
    data_extraction = df[col]
    data_extraction.columns = col_rename
    
    for i in range(0,len(col2)-1):
        data_extraction[col2[i]]=data_extraction[col_rename[2*i]]/data_extraction[col_rename[2*i+1]]
    result = data_extraction[col2]
    return result

In [39]:
def split_theta_alpha_data(df, label, end):
    data_extraction = cal_theta_alpha(df)
    rest = data_extraction[data_extraction.vis_name == 0].dropna(axis=0)
    survey = data_extraction[data_extraction.vis_name == 100].dropna(axis=0)
    
    vis = data_extraction[data_extraction.vis_name == 1].reset_index(drop=True).dropna(axis=0)
    vis['label'] = label[0]
    vis.drop(['vis_name'], axis=1, inplace=True)
    
    for i in range(2,end):
        df = data_extraction[data_extraction.vis_name == i].reset_index(drop=True)
        df['label'] = label[i-1]
        df.drop(['vis_name'], axis=1, inplace=True)
        df = df.dropna(axis=0)
        vis = pd.concat([vis,df], ignore_index=True, axis=0)
    return rest, survey, vis

In [ ]:
cnt = 0
for eeg, label in zip (eegData, workloadLevel):
    if cnt == 0 :
        rest, survey, vis = split_theta_alpha_data(eeg, label,21)
        cnt+=1
        continue
    elif cnt == 6:
        r, s, v = split_theta_alpha_data(eeg, label,21)
    else:
        r, s, v = split_theta_alpha_data(eeg, label, 22)
    
    rest = pd.concat([rest,r], ignore_index=True, axis=0)
    survey = pd.concat([survey,s], ignore_index=True, axis=0)
    vis = pd.concat([vis,v], ignore_index=True, axis=0)
    cnt+=1

In [41]:
vis

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,label
0,2.952833,3.105452,1.733649,1.286172,0.997657,1.123533,1.104817,1.036953,1.283992,1.757355,1.490719,1.281464,3.378006,2.528323,6
1,3.337757,3.067987,1.808938,1.269267,0.947180,1.063313,1.139223,0.986136,1.240326,1.668122,1.543780,1.290260,3.529764,2.448986,6
2,3.460255,2.889841,1.808787,1.317058,1.021541,1.094298,1.247150,1.009595,1.237751,1.655340,1.565374,1.313119,3.339320,2.115198,6
3,3.400069,2.708860,1.791900,1.424313,1.191386,1.205574,1.426901,1.120465,1.306624,1.768321,1.586444,1.378863,2.988018,1.751219,6
4,3.241047,2.532158,1.808551,1.631368,1.419774,1.379464,1.662086,1.299213,1.425712,1.995887,1.627646,1.480259,2.582288,1.432403,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108207,3.587874,4.357214,2.334554,2.569024,2.588199,0.650938,1.722900,1.677678,3.284874,77.501610,2.656741,2.109043,5.051498,4.562173,5
108208,3.698659,4.904903,2.268399,2.620445,3.133342,0.698348,1.671146,1.395313,3.428622,68.175779,2.429011,1.763087,4.876154,4.746517,5
108209,3.660792,5.359041,2.181249,2.608695,4.296135,0.733687,1.795107,1.197751,3.339025,62.193847,2.148419,1.510463,4.708856,4.880381,5
108210,3.514900,5.855391,2.066605,2.640225,5.978087,0.750709,2.052408,1.085270,3.014408,61.070530,1.819241,1.322830,4.551947,4.934379,5


In [43]:
label=vis['label']
label = to_categorical(label,11)
data=vis.drop(['label'],axis=1)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)
data = pd.DataFrame(scaled, columns = data.columns, index=data.index)
data
print(data.shape, label.shape)

(108212, 14) (108212, 11)


In [44]:
def windowing_dataset(data, label, window_size):
    data_list = []
    label_list = []
    
    for i in range(0,len(data)//window_size,window_size):
        data_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i]))
    return np.array(data_list), np.array(label_list)

In [45]:
x, y = windowing_dataset(data,pd.DataFrame(label),14)
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.6, 
                                                    random_state=True,
                                                    stratify = y)

print(x_train.shape, x_test.shape,y_train.shape, y_test.shape)

(331, 14, 14) (222, 14, 14) (331, 11) (222, 11)


In [11]:
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

def drawResult(history):
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(history.history['loss'], 'y', label='train loss')
    loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')

    acc_ax.plot(history.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(history.history['val_accuracy'], 'g', label='val acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='upper left')

    plt.show()
    
def runModel(model, crossentropy, x_train, y_train, x_valid, y_valid):
    early_stop = EarlyStopping(monitor='val_loss', patience=10)
    model.compile(optimizer='adam', loss= crossentropy, metrics='accuracy')

    history = model.fit(x_train, y_train, 
                        epochs = 200, 
                        validation_data = (x_valid, y_valid), 
                        callbacks=[early_stop], verbose=1)
    drawResult(history)
    return model

def eval_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    return f1_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')

In [10]:
def Model2_2_1():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dropout(0.25))
    
    model.add(Dense(11, activation='sigmoid'))
    
    #model.summary()
    return model

def Model2_2_2():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(2,2), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(2,2), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(128, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(32, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(32, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dropout(0.25))
    
    model.add(Dense(50, activation='relu'))
    model.add(Dense(11, activation='sigmoid'))
    
    model.summary()
    return model

def Model2_2_3():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(128, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(32, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(32, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(11, activation='sigmoid'))
    
    #model.summary()
    return model

crossentropy = 'categorical_crossentropy'

print(eval_model(runModel(Model2_2_1(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))
print(eval_model(runModel(Model2_2_2(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))
print(eval_model(runModel(Model2_2_3(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))

In [ ]:
def Model2_1_1():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dropout(0.25))
    
    model.add(Dense(11, activation='sigmoid'))
    
    #model.summary()
    return model

def Model2_1_2():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(2,2), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(2,2), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(128, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(32, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(32, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dropout(0.25))
    
    model.add(Dense(50, activation='relu'))
    model.add(Dense(11, activation='sigmoid'))
    
    #model.summary()
    return model

def Model2_1_3():
    model = Sequential()
    model.add(InputLayer(input_shape=(14, 14)))
    model.add(ExpandLastDim())
    model.add(CastToFloat32())
    
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(Conv2D(512, kernel_size=(4,4), strides=(1,1), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(128, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Conv2D(32, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    model.add(Conv2D(32, kernel_size=(4,4), strides=(1,1), activation='relu', padding='same'))
    
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(11, activation='sigmoid'))
    
    #model.summary()
    return model

crossentropy = 'categorical_crossentropy'

print(eval_model(runModel(Model2_1_1(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))
print(eval_model(runModel(Model2_1_2(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))
print(eval_model(runModel(Model2_1_3(),crossentropy,
                          x_train, y_train, x_valid, y_valid), x_test, y_test))

# AUTO KERAS

In [29]:
import autokeras as ak
from tensorflow.keras.utils import plot_model
from datetime import datetime

In [45]:
x_fft, y_fft = windowing_dataset(data_fft,pd.DataFrame(label_fft),10)
print(x_fft.shape, y_fft.shape)
x_train, x_test, y_train, y_test = train_test_split(x_fft,y_fft, train_size=0.6, 
                                                    random_state=True,
                                                    stratify = y_fft)

print(x_train.shape, x_test.shape,y_train.shape, y_test.shape)

(1083, 10, 70) (1083, 11)
(649, 10, 70) (434, 10, 70) (649, 11) (434, 11)


In [ ]:
trials=[3] #,5,10,20]
for trial in trials:
    clf_ = ak.ImageClassifier(overwrite=True, max_trials=trial)
    clf_.fit(x=x_train, y=y_train, epochs=20)

    predicted_y = clf_.predict(x_test)
    print(predicted_y)
    loss, acc = clf_.evaluate(x_test, y_test)
    print(clf_.evaluate(x_test, y_test))
    print('Loss: %.3f   Accuracy: %.3f' % (loss,acc))

    model = clf_.export_model()
    model.summary()
    plot_model(model, show_shapes=True)
    tmp = int(acc*100)
    print(tmp)
    model.save('model/'+'_ACC_'+str(tmp)+'_try_'+str(trial)+'_.h5')

Trial 1 Complete [00h 00m 06s]
val_loss: 0.955339252948761

Best val_loss So Far: 0.955339252948761
Total elapsed time: 00h 00m 06s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
image_block_1/b...|resnet            |vanilla           
image_block_1/n...|True              |True              
image_block_1/a...|True              |False             
image_block_1/i...|True              |None              
image_block_1/i...|True              |None              
image_block_1/i...|0                 |None              
image_block_1/i...|0                 |None              
image_block_1/i...|0.1               |None              
image_block_1/i...|0                 |None              
image_block_1/r...|False             |None              
image_block_1/r...|resnet50          |None              
image_block_1/r...|True              |None              
classification_...|global_avg        |flatten           
classification_...|0                 |0.5  